In [1]:
pip install pinecone cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.7 MB/s eta 0:00:00


In [13]:
import cohere
from pinecone import Pinecone, ServerlessSpec
import pandas as pd

In [4]:
def initialize_cohere(api_key):
    """
    Initialize the Cohere client.

    Args:
    api_key (str): Your Cohere API key.

    """
    return cohere.Client(api_key)

In [21]:
def initialize_pinecone(api_key, environment, index_name, dimension):
    """
    Initialize the Pinecone vector database and create an index if it doesn't exist.

    Args:
    api_key (str): Your Pinecone API key.
    environment (str): Pinecone environment region.
    index_name (str): Name of the Pinecone index.
    dimension (int): Dimension of the embeddings.

    Returns:
    pinecone.Index: The initialized Pinecone index.
    """
    pc = Pinecone(api_key=api_key)
    if index_name not in [idx.name for idx in pc.list_indexes()]:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric='cosine',
            spec=ServerlessSpec(
                cloud=environment,
                region='us-east-1'
            )
        )
    return pc.Index(index_name)

In [27]:
# Generate embedding for text using Cohere
def generate_embedding(cohere_client, text, model="embed-english-v3.0"):
    """
    Generate an embedding for the given text using Cohere.

    Args:
    cohere_client (cohere.Client): The Cohere client.
    text (str): The input text to embed.
    model (str): The embedding model to use.

    Returns:
    list: The embedding vector.
    """
    response = cohere_client.embed(texts=[text],input_type="search_query",model=model)
    return response.embeddings[0]

In [8]:
# Store embeddings in the Pinecone vector database
def store_embeddings(index, dataset, cohere_client):
    """
    Store embeddings and metadata in the Pinecone vector database.

    Args:
    index (pinecone.Index): The Pinecone index.
    dataset (list of dict): Dataset containing 'question', 'prompt', 'answer', and 'priority'.
    cohere_client (cohere.Client): The Cohere client.
    """
    vectors = []
    for idx, row in dataset.iterrows():
        question = row['question']
        metadata = {
            "question": row['question'],  #Added to metadata for easier retrieval
            "prompt": row['prompt'],
            "answer": row['answer'],
            "strategy": row['strategy'],
            "priority": int(row['priority'])  #ensure priority is stored as an integer
        }
        embedding = generate_embedding(cohere_client, question)
        vectors.append((f"q{idx}", embedding, metadata))
    index.upsert(vectors)

In [29]:
def retrieve_top_k_questions(index, cohere_client, new_question, top_k=5):
    """
    Retrieve the top-K most similar questions for a given new question.

    Args:
    index (pinecone.Index): The Pinecone index.
    cohere_client (cohere.Client): The Cohere client.
    new_question (str): The input question.
    top_k (int): Number of top similar questions to retrieve.

    Returns:
    list of dict: Retrieved questions with metadata and relevance scores.
    """
    # Generate embedding for the new question
    query_embedding = generate_embedding(cohere_client, new_question)

    # Query the vector database
    results = results = index.query(vector=query_embedding,top_k=5,include_metadata=True
)

    # Extract and format results
    retrieved_questions = []
    for match in results['matches']:
        retrieved_questions.append({
            "question": match['metadata'].get('question', 'Unknown Question'),
            "prompt": match['metadata'].get('prompt'),
            "answer": match['metadata'].get('answer'),
            "strategy": match['metadata'].get('strategy'),
            "priority": match['metadata'].get('priority'),
            "score": match['score']
        })

    return retrieved_questions

In [30]:
def main():
    # Initialize clients
    cohere_api_key = "QSVeHnX7B2cUcbZpbQFGpIDA4zlOcSWGyQ74wk3v"  # Replace with your Cohere API key
    pinecone_api_key = "pcsk_2Aovig_CsVHa6sDHrnxX9YM97Ud1EsGW7hcvF1sTy3DNBMhvcouxrtAARup5mACrpMkAAe"  # Replace with your Pinecone API key
    pinecone_env = "aws"
    index_name = "ap-retrieval"
    embedding_dim = 1024

    cohere_client = initialize_cohere(cohere_api_key)
    index = initialize_pinecone(pinecone_api_key, pinecone_env, index_name, embedding_dim)

    csv_file_path = "/content/gsm8k_prompt_dataset.csv"  # Replace with your file path
    dataset = pd.read_csv(csv_file_path)

    # Store embeddings
    store_embeddings(index, dataset, cohere_client)

    # Retrieve strategies for a new question
    new_question = "A chef has 500 cupcakes. She wants to distribute them equally into 10 boxes. How many cupcakes will each box contain?"
    results = retrieve_top_k_questions(index, cohere_client, new_question)

    # Print the results
    print("Retrieved Strategies:")
    for strategy in results:
        print(f"Question: {strategy['question']}, "
              f"Prompt: {strategy['prompt']}, "
              f"Strategy: {strategy['strategy']}, "
              f"Priority: {strategy['priority']}, "
              f"Score: {strategy['score']}")

if __name__ == "__main__":
    main()


Retrieved Strategies:
Question: A baker has 240 cookies. He wants to divide them equally among 8 boxes. How many cookies will each box have?, Prompt: Q: A baker has 240 cookies. He wants to divide them equally among 8 boxes. How many cookies will each box have? A: 240 ÷ 8 = 30. The Final Answer is 30., Strategy: Zero-Shot, Priority: 1.0, Score: 0.635134518
Question: A baker has 240 cookies. He wants to divide them equally among 8 boxes. How many cookies will each box have?, Prompt: Q: A baker has 240 cookies. He wants to divide them equally among 8 boxes. A: Let’s calculate step by step. Divide 240 by 8. The Final Answer is 30., Strategy: CoT, Priority: 3.0, Score: 0.635134518
Question: Sarah bought 3 packs of pens. Each pack has 10 pens. How many pens did Sarah buy in total?, Prompt: Q: Sarah bought 3 packs of pens. Each pack has 10 pens. How many pens did Sarah buy in total? A: 3 × 10 = 30. The Final Answer is 30., Strategy: Zero-Shot, Priority: 1.0, Score: 0.461693525
Question: Lisa